In [3]:
import os
import numpy as np

In [4]:
class Problem_formulation:
    def __init__(self,initial, goal =np.array([[1,2,3],[4,5,6],[7,8,0]])):
        self.init_state=initial
        self.goal = goal
        
    def goal_test(self, state):
        check_goal= np.equal(state, self.goal).all()
        return check_goal
  #Find possible actions
    def p_action(self, state):
        self.p_act = []
        self.state=state
        #Position of zero in state
        self.p_zero = np.where(self.state == 0)
        #print('p_zero',self.p_zero)
        if (self.p_zero[1] == 1):
            self.p_act.append('R')
            self.p_act.append('L')
        elif self.p_zero[1] == 0: self.p_act.append('L')
        elif self.p_zero[1] == 2: self.p_act.append('R')
    
        if (self.p_zero[0] == 1):
            self.p_act.append('D')
            self.p_act.append('U')
        elif self.p_zero[0] == 2: self.p_act.append('D')
        elif self.p_zero[0] == 0: self.p_act.append('U')
        return self.p_act
    
    def trans(self, state, act):
        self.c_state = np.copy(state)
        self.p_zero = np.where(self.c_state  == 0)
        if act=='D':
            self.c_state = np.copy(state)
            self.c_state[self.p_zero[0],self.p_zero[1]] = self.c_state[self.p_zero[0]-1,self.p_zero[1]]
            self.c_state[self.p_zero[0]-1,self.p_zero[1]] = 0
            self.p_state=self.c_state
            return self.p_state
        elif act=='R':
            self.c_state = np.copy(state)
            self.c_state[self.p_zero[0],self.p_zero[1]] = self.c_state[self.p_zero[0],self.p_zero[1]-1]
            self.c_state[self.p_zero[0],self.p_zero[1]-1] = 0
            self.p_state=self.c_state
            return self.p_state
        elif act=='L':
            self.c_state = np.copy(state)
            self.c_state[self.p_zero[0],self.p_zero[1]] = self.c_state[self.p_zero[0],self.p_zero[1]+1]
            self.c_state[self.p_zero[0],self.p_zero[1]+1] = 0 
            self.p_state=self.c_state
            return self.p_state
        elif act=='U':
            self.c_state = np.copy(state)
            self.c_state[self.p_zero[0],self.p_zero[1]] = self.c_state[self.p_zero[0]+1,self.p_zero[1]]
            self.c_state[self.p_zero[0]+1,self.p_zero[1]] = 0
            self.p_state=self.c_state
            return self.p_state

In [5]:
class Node:
    def __init__(self, state,parent= None, action=None, depth=0):
        self.state= state
        self.parent= parent
        self.action = action
        self.depth = depth 

    def get_parent(self):
        return self.parent
  
    def get_path(self):
        self.paths=[]
        next_node = self.get_parent()
        cur_action = self.action
        while next_node:
            self.paths.append(cur_action)
            cur_action = next_node.action
            next_node = next_node.get_parent()
        return self.paths

In [97]:
class Run_puzzle:
    def __init__(self, file_path, search_name):
        self.file_path = file_path
        self.search_name = search_name
        print("filename", file_path)
        print("search", search_name)
  
  #read initial state from file path
    def read_file(self):
        if isinstance(self.file_path,str):
            init_state= []
            with open(self.file_path,'r') as f:
                for line in f:
                    line=line.replace("\n","")
                    if '_' in line:
                        line=line.replace("_",'0')
                    _line=line.split(' ')
                    for i in _line:
                        init_state.append(eval(i)) 
                init_state=np.array(init_state)
                init_state= np.reshape(init_state,(3,3))

                return init_state
        ###example case - directly input array in order to test easy case
        else:
            init_state= self.file_path
            return init_state

    def solvable(self, state):
        #whether is solvable or not
        _flat_stat=state.flatten()
        n_inv=0
        ##Ignore blank space

        zero_pos=np.argwhere(_flat_stat==0)
        no_zero_stat=np.delete(_flat_stat,zero_pos)
        _len=0
        for i in range(1,len(no_zero_stat)):
              _len+=np.count_nonzero(no_zero_stat[i:][no_zero_stat[i:]<no_zero_stat[i-1]])
        return _len%2==0

    def run_game(self):
        # 1.load problem formulation
        init_state = self.read_file()
        # 2.Check solvable
        self.b_sol = self.solvable(init_state)
        if not(self.b_sol):
            print("--current state--")
            print(init_state)
            return print("Not solvable")

        problem=Problem_formulation(init_state)
        print("search name",self.search_name)
        if self.search_name =='bfs':
            #Search class generated
            result=self.BFS(problem)
            return result
        
        elif self.search_name =='dfs':
            #Search class generated
            print("d")
            result=self.DFS(problem)
            return result
        
        elif self.search_name =='h1':
            result=self.h1_A_star(problem)
            return result
        
    def check_visited(self,_cur_state,_que):
        for _idx,_item in enumerate(_que):
            if np.equal(_cur_state,_item[0]).all():
                return False
        return True

    def track(self, problem, _final_state, _act):
        _cu = _final_state
        _path=[]
        
        while not(np.equal(_cu,problem.init_state).all()):
            _path.append(_act)
            if(_act == 'U'):
                _parent=problem.trans(_cu,'D')
            elif(_act == 'D'):
                _parent=problem.trans(_cu,'U')
            elif(_act == 'R'):
                _parent=problem.trans(_cu,'L')
            elif(_act == 'L'):
                _parent=problem.trans(_cu,'R')
            _cu=_parent
        
        return _path
            
    def BFS(self, problem):
        print("BFS")
        #print(problem.init_state)
        
        dep=0
        closed_queue=[]
        opened_queue=[(problem.init_state,None,0)]

        while opened_queue:
            #Starting exploring
            ##pop opened queue
            
            cur_state, cur_act, cur_dep = opened_queue.pop(0)
            closed_queue.append((cur_state,cur_act,dep))
            print(cur_dep)
            if(problem.goal_test(cur_state)):
                print("Find")
                print(len(opened_queue))
                print(len(closed_queue))
                #bfs_path = closed_queue[-1].get_path()
                #bfs_depth = closed_queue[-1].depth
                #print(bfs_path)
                #path=self.track(problem, cur_state, cur_act)
                print(cur_dep)
                return closed_queue+opened_queue
            
            for _act in problem.p_action(cur_state):
                _s = problem.trans(cur_state, _act)
                if(self.check_visited(_s, closed_queue)) and (self.check_visited(_s, opened_queue)):
                    opened_queue.append((_s,_act,cur_dep+1))
                    print("appended")
                    print(_s,_act)
                    #print("depth",cur_dep+1)
    
    def DFS(self, problem):
        print("DFS")
        #print(problem.init_state)
        
        dep=0
        closed_queue=[]
        opened_queue=[(problem.init_state,None,0)]

        while opened_queue:
            #Starting exploring
            ##pop opened queue
            
            cur_state, cur_act, cur_dep = opened_queue.pop(-1)
            closed_queue.append((cur_state,cur_act,dep))
            print(cur_dep)
            if(problem.goal_test(cur_state)):
                print("Find")
                print(len(opened_queue))
                print(len(closed_queue))
                #bfs_path = closed_queue[-1].get_path()
                #bfs_depth = closed_queue[-1].depth
                #print(bfs_path)
                #path=self.track(problem, cur_state, cur_act)
                print(cur_dep)
                return closed_queue+opened_queue
            
            for _act in problem.p_action(cur_state):
                _s = problem.trans(cur_state, _act)
                if(self.check_visited(_s, closed_queue)) and (self.check_visited(_s, opened_queue)):
                    opened_queue.append((_s,_act,cur_dep+1))
                    print("appended")
                    print(_s,_act)
                    #print("depth",cur_dep+1)
        
    def h1_A_star(self, problem):
        print("h1_A_star")
        #print(problem.init_state)

        dep=0
        closed_queue=[]
        #f=h+dep(g)

        ##Heuristic
        _h = self.heuristic_h1(problem.init_state,problem)
        f = _h+ dep
        opened_queue=[(problem.init_state,None,0,f)]
        
        while opened_queue:
            #Starting exploring
            ##pop opened queue

            #find idx
            opened_queue.sort(key=self.sort_by_heu)
            print(opened_queue)
            cur_state, cur_act, cur_dep,f = opened_queue.pop(0)



            closed_queue.append((cur_state,cur_act,dep,f))
            print(cur_dep)
            if(problem.goal_test(cur_state)):
                print("Find")
                print(len(opened_queue))
                print(len(closed_queue))
                #bfs_path = closed_queue[-1].get_path()
                #bfs_depth = closed_queue[-1].depth
                #print(bfs_path)
                #path=self.track(problem, cur_state, cur_act)
                print(cur_dep)
                return closed_queue+opened_queue

            for _act in problem.p_action(cur_state):
                _s = problem.trans(cur_state, _act)
                if(self.check_visited(_s, closed_queue)) and (self.check_visited(_s, opened_queue)):

                    _h = self.heuristic_h1(cur_state,problem)
                    f = _h+ dep
                    opened_queue.append((_s,_act,cur_dep+1,f+1))
                    print("appended")
                    print(_s,_act)
                    #print("depth",cur_dep+1)
                    
    def heuristic_h1(self, state, problem):
        goal= problem.goal
        _cur_s = state.copy()
        _cur_s = _cur_s.flatten()
        _goal = goal.flatten()

        _c = 0
        for _idx, _s in enumerate(_cur_s):
            if _s !=0 and _s != _goal[_idx]:
                _c=_c+1
        return _c

    def sort_by_heu(self,val):
        return val[3]

l.sort(key=sortSecond)
print(l)
                        
"""
print(len(opened_queue))
print("current\n",cur_node.state)
if cur_node.parent:
print("parent",cur_node.parent.state)
"""

[(1, 3, 0), (1, 1, 3), (1, 2, 4)]


'\nprint(len(opened_queue))\nprint("current\n",cur_node.state)\nif cur_node.parent:\nprint("parent",cur_node.parent.state)\n'

In [98]:
##main##
a=Run_puzzle('S5.txt','h1')
_queues=a.run_game()

filename S5.txt
search h1
search name h1
h1_A_star
[(array([[4, 1, 3],
       [2, 0, 6],
       [7, 5, 8]]), None, 0, 5)]
0
appended
[[4 1 3]
 [0 2 6]
 [7 5 8]] R
appended
[[4 1 3]
 [2 6 0]
 [7 5 8]] L
appended
[[4 0 3]
 [2 1 6]
 [7 5 8]] D
appended
[[4 1 3]
 [2 5 6]
 [7 0 8]] U
[(array([[4, 1, 3],
       [0, 2, 6],
       [7, 5, 8]]), 'R', 1, 6), (array([[4, 1, 3],
       [2, 6, 0],
       [7, 5, 8]]), 'L', 1, 6), (array([[4, 0, 3],
       [2, 1, 6],
       [7, 5, 8]]), 'D', 1, 6), (array([[4, 1, 3],
       [2, 5, 6],
       [7, 0, 8]]), 'U', 1, 6)]
1
appended
[[0 1 3]
 [4 2 6]
 [7 5 8]] D
appended
[[4 1 3]
 [7 2 6]
 [0 5 8]] U
[(array([[4, 1, 3],
       [2, 6, 0],
       [7, 5, 8]]), 'L', 1, 6), (array([[4, 0, 3],
       [2, 1, 6],
       [7, 5, 8]]), 'D', 1, 6), (array([[4, 1, 3],
       [2, 5, 6],
       [7, 0, 8]]), 'U', 1, 6), (array([[0, 1, 3],
       [4, 2, 6],
       [7, 5, 8]]), 'D', 2, 6), (array([[4, 1, 3],
       [7, 2, 6],
       [0, 5, 8]]), 'U', 2, 6)]
1
appended
[[4 1 